In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('homework05') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/03/22 23:02:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.version

'3.3.2'

In [10]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-03-22 21:54:37--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.154.99.220, 18.154.99.47, 18.154.99.177, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.154.99.220|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M   270MB/s    in 0.2s    

2025-03-22 21:54:38 (270 MB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



In [4]:
!ls -lh

total 780M
-rw-rw-r-- 1 sidd4ml sidd4ml 7.7K Mar 16 19:16 03_test.ipynb
-rw-rw-r-- 1 sidd4ml sidd4ml  19K Mar 16 19:54 04_pyspark.ipynb
-rw-rw-r-- 1 sidd4ml sidd4ml  22K Mar 16 21:49 04_pyspark_excercise.ipynb
-rw-rw-r-- 1 sidd4ml sidd4ml  25K Mar 17 21:41 05_taxi_schema.ipynb
-rw-rw-r-- 1 sidd4ml sidd4ml 8.7K Mar 17 21:53 06_spark_sql.ipynb
-rw-rw-r-- 1 sidd4ml sidd4ml 2.5K Feb  6 19:58 06_spark_sql.py
-rw-rw-r-- 1 sidd4ml sidd4ml 2.6K Feb  6 19:58 06_spark_sql_big_query.py
-rw-rw-r-- 1 sidd4ml sidd4ml 7.6K Feb  6 19:58 07_groupby_join.ipynb
-rw-rw-r-- 1 sidd4ml sidd4ml  12K Feb  6 19:58 08_rdds.ipynb
-rw-rw-r-- 1 sidd4ml sidd4ml 4.3K Feb  6 19:58 09_spark_gcs.ipynb
-rw-rw-r-- 1 sidd4ml sidd4ml  18K Mar 22 23:02 Homework05.ipynb
-rw-rw-r-- 1 sidd4ml sidd4ml 3.0K Feb  6 19:58 cloud.md
drwxrwxr-x 4 sidd4ml sidd4ml 4.0K Mar 17 21:36 data
-rwxrwxr-x 1 sidd4ml sidd4ml  801 Mar 18 00:20 download_data.sh
-rw-rw-r-- 1 sidd4ml sidd4ml 718M Jul 14  2022 fhvhv_tripdata_2021-01.csv
-rw-rw-r-- 1 s

In [5]:
!wc -l yellow_tripdata_2024-10.parquet

254534 yellow_tripdata_2024-10.parquet


In [6]:
df = spark.read\
.option("header", "true") \
.parquet('yellow_tripdata_2024-10.parquet')

In [7]:
df.count()

3833771

In [8]:
df.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

In [64]:
df.schema

StructType([StructField('VendorID', IntegerType(), True), StructField('tpep_pickup_datetime', TimestampType(), True), StructField('tpep_dropoff_datetime', TimestampType(), True), StructField('passenger_count', LongType(), True), StructField('trip_distance', DoubleType(), True), StructField('RatecodeID', LongType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('payment_type', LongType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('Airport_fee', DoubleType(), True)])

In [65]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [22]:
df = df.repartition(4)

In [24]:
df.write.parquet('data/pq/yellow/2024/10/')

In [11]:
!ls -lh data/pq/yellow/2024/10/

total 97M
-rw-r--r-- 1 sidd4ml sidd4ml   0 Mar 22 22:32 _SUCCESS
-rw-r--r-- 1 sidd4ml sidd4ml 25M Mar 22 22:32 part-00000-1e1e956c-9f2d-4640-94d4-2b7502213f9c-c000.snappy.parquet
-rw-r--r-- 1 sidd4ml sidd4ml 25M Mar 22 22:32 part-00001-1e1e956c-9f2d-4640-94d4-2b7502213f9c-c000.snappy.parquet
-rw-r--r-- 1 sidd4ml sidd4ml 25M Mar 22 22:32 part-00002-1e1e956c-9f2d-4640-94d4-2b7502213f9c-c000.snappy.parquet
-rw-r--r-- 1 sidd4ml sidd4ml 25M Mar 22 22:32 part-00003-1e1e956c-9f2d-4640-94d4-2b7502213f9c-c000.snappy.parquet


In [66]:
df.registerTempTable('taxi_trips')

In [67]:
spark.sql("""
SELECT 
    COUNT(*) 
FROM 
   taxi_trips
WHERE 
   tpep_pickup_datetime BETWEEN '2024-10-15 00:00:00' AND '2024-10-15 23:59:59'
""").show()

+--------+
|count(1)|
+--------+
|  128893|
+--------+



In [68]:
spark.sql("""
SELECT 
    (unix_timestamp(tpep_dropoff_datetime) - unix_timestamp(tpep_pickup_datetime)) / 3600.0 
        AS trip_duration_hours
FROM 
    taxi_trips 
ORDER BY 
    trip_duration_hours DESC
""").show(5)

+-------------------+
|trip_duration_hours|
+-------------------+
|         162.617778|
|         143.325000|
|         137.760556|
|         114.834722|
|          89.898333|
+-------------------+
only showing top 5 rows



In [45]:
!mkdir -p data/raw/zones

In [46]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv -O data/raw/zones/taxi_zone_lookup.csv

--2025-03-23 03:58:51--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.154.99.220, 18.154.99.177, 18.154.99.47, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.154.99.220|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘data/raw/zones/taxi_zone_lookup.csv’

data/raw/zones/taxi 100%[===================>]  12.04K  --.-KB/s    in 0.001s  

2025-03-23 03:58:51 (15.0 MB/s) - ‘data/raw/zones/taxi_zone_lookup.csv’ saved [12331/12331]



In [53]:
df_zones = spark.read\
.option("header", "true") \
.csv('data/raw/zones/taxi_zone_lookup.csv')

In [54]:
df_zones.schema

StructType([StructField('LocationID', StringType(), True), StructField('Borough', StringType(), True), StructField('Zone', StringType(), True), StructField('service_zone', StringType(), True)])

In [55]:
df_zones.printSchema()

root
 |-- LocationID: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [63]:
df_zones.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [70]:
df_zones.registerTempTable('taxi_zone_lookup')

In [73]:
spark.sql("""
SELECT 
    a.PULocationID, 
    b.Zone,
    COUNT(*) AS trip_count
FROM 
    taxi_trips a
INNER JOIN 
    taxi_zone_lookup b
    ON a.PULocationID = b.LocationID
GROUP BY 
    a.PULocationID, 
    b.Zone
ORDER BY 
    trip_count ASC
""").show(5)

+------------+--------------------+----------+
|PULocationID|                Zone|trip_count|
+------------+--------------------+----------+
|         105|Governor's Island...|         1|
|           5|       Arden Heights|         2|
|         199|       Rikers Island|         2|
|         111| Green-Wood Cemetery|         3|
|           2|         Jamaica Bay|         3|
+------------+--------------------+----------+
only showing top 5 rows



#### ALternative Way

In [74]:
from pyspark.sql import functions as F

In [75]:
# Join on PULocationID (from trips) = LocationID (from zones)
df_join = df.join(
    df_zones, 
    df.PULocationID == df_zones.LocationID, 
    how='inner'
)

In [77]:
df_join.registerTempTable('trips_with_locationName')

In [78]:
df_join.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)
 |-- LocationID: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone

In [82]:
spark.sql("""
SELECT 
    Zone AS PICKUP_LOCATION_ZONE,
    COUNT(*) AS trip_count
FROM 
trips_with_locationName
GROUP BY PICKUP_LOCATION_ZONE
ORDER BY trip_count ASC
""").show(5)

+--------------------+----------+
|PICKUP_LOCATION_ZONE|trip_count|
+--------------------+----------+
|Governor's Island...|         1|
|       Rikers Island|         2|
|       Arden Heights|         2|
| Green-Wood Cemetery|         3|
|         Jamaica Bay|         3|
+--------------------+----------+
only showing top 5 rows

